In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

2024-10-24 20:59:58.966423: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 20:59:59.035947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 20:59:59.075004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 20:59:59.087564: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 20:59:59.139107: I tensorflow/core/platform/cpu_feature_guar

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [4]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z, res.visibility] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*4)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [5]:
import warnings
def extractPostitionFromVideo(filepath, number_of_frames=30):
    try:
        cap = cv2.VideoCapture(filepath)
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        num_frames_to_extract = number_of_frames
        
        if num_frames_to_extract > total_frames:
            warnings.warn(f"Requested {num_frames_to_extract} frames, but the video only has {total_frames} frames. Defaulting to max.")
            num_frames_to_extract = total_frames 
        
        frame_interval = total_frames // num_frames_to_extract
        
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            frame_num = 0
            extracted_frame_count = 0
            keypoints_list = []

            while cap.isOpened() and extracted_frame_count < num_frames_to_extract:
                ret, frame = cap.read()
                
                if not ret:
                    break
                
                # Only process frames at the defined interval
                if frame_num % frame_interval == 0:
                    image, results = mediapipe_detection(frame, holistic)
                    keypoints = extract_keypoints(results)
                    keypoints_list.append(keypoints)
                    
                    # Increment extracted frame count
                    extracted_frame_count += 1

                frame_num += 1

        cap.release()

        # Optionally save the keypoints list as a .npy file or process as needed
        return keypoints_list

    except Exception as e:
        print(e)
        cap.release()

In [13]:
exp = np.array(extractPostitionFromVideo('Training Data/Are you free today/20241004_1111140000-0130.mp4'))

W0000 00:00:1729783884.322110   55916 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729783884.724360   55916 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729783884.726958   55917 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729783884.726975   55912 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729783884.727576   55914 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729783884.785581   55914 inference_feedback_manager.cc:114] Feedback manager 

In [14]:
exp = exp[np.newaxis, ...]

In [8]:
import keras

model = keras.models.load_model('VideoRecognitionV2_Bekar.keras')

I0000 00:00:1729783816.267565   55118 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729783816.554304   55118 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729783816.554418   55118 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729783816.556828   55118 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729783816.556866   55118 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [15]:
pred = model.predict(exp)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [16]:
pred

array([[1.0000000e+00, 1.8699620e-25, 1.3993218e-10, 2.1530540e-12,
        7.7873843e-16, 2.3238449e-16, 1.2180029e-22, 2.0602130e-14,
        1.8611962e-08, 5.8334883e-29, 1.1483754e-10, 2.9594190e-27,
        1.1613791e-16, 1.5573773e-14, 5.1832067e-30]], dtype=float32)

In [17]:
actions = np.array(os.listdir('Training Data'))

In [18]:
actions[np.argmax(pred)]

'Are you free today'